# Combine video files 

In this notebook we will be combining video files into a single video file. The below code  will combine 3 video files into 1 video in parallel frame aligning timestamp for all three video in parallel

# Install libraries

In [ ]:
pip install opencv-python numpy

In [ ]:
pip install opencv-python-headless

In [ ]:
pip install ffmpeg-python

In [ ]:
import cv2
print(cv2.__version__)


# Initialize the variables

In [ ]:
video1_path = "data/video1.mov"
video2_path = "data/video1.mov"
video3_path = "data/video1.mov"
output_path = "data/combined_video.mov"

# Stitch the videos together

In [ ]:
import cv2
import numpy as np
import ffmpeg
from collections import defaultdict
import os
import tempfile
import shutil

def verify_videos(video1, video2, video3):
    """Verify that the video paths are different and files exist"""
    videos = [video1, video2, video3]
    if len(set(videos)) != 3:
        raise ValueError("All video paths must be different!")
    
    for i, video in enumerate(videos, 1):
        if not os.path.exists(video):
            raise FileNotFoundError(f"Video {i} not found: {video}")
        
    print("Video paths verified:")
    for i, video in enumerate(videos, 1):
        print(f"Video {i}: {video}")
    print()

def combine_videos_parallel(video1, video2, video3, output_video):
    # First verify the videos
    verify_videos(video1, video2, video3)
    
    # Create VideoCapture objects
    cap1 = cv2.VideoCapture(video1)
    cap2 = cv2.VideoCapture(video2)
    cap3 = cv2.VideoCapture(video3)
    
    # Store video captures in a dictionary
    caps = {
        1: {'cap': cap1, 'path': video1},
        2: {'cap': cap2, 'path': video2},
        3: {'cap': cap3, 'path': video3}
    }

    # Validate that videos opened successfully and print properties
    for idx, cap_info in caps.items():
        cap = cap_info['cap']
        path = cap_info['path']
        
        if not cap.isOpened():
            raise RuntimeError(f"Error: Cannot open video {idx}: {path}")
        
        # Print video properties for debugging
        print(f"Video {idx} properties ({path}):")
        print(f"Width: {int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))}")
        print(f"Height: {int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")
        print(f"FPS: {cap.get(cv2.CAP_PROP_FPS)}")
        print(f"Frame count: {int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}\n")

        # Read first frame to verify content
        ret, frame = cap.read()
        if not ret:
            raise RuntimeError(f"Error: Cannot read frames from video {idx}: {path}")
        # Reset video to start
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

    # Get video properties
    width = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = min(cap_info['cap'].get(cv2.CAP_PROP_FPS) for cap_info in caps.values())

    # Create temporary directory for frames
    temp_dir = tempfile.mkdtemp()
    frame_count = 0

    try:
        while True:
            # Read frames from each video
            frames = {}
            all_ended = True
            
            for idx, cap_info in caps.items():
                ret, frame = cap_info['cap'].read()
                if ret:
                    all_ended = False
                    frames[idx] = frame.copy()
                    # Add debug text to identify source video
                    cv2.putText(frames[idx], f"Video {idx} - {os.path.basename(cap_info['path'])}", 
                              (10, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 
                              0.7, (0, 255, 0), 2)

            if all_ended:
                break

            if len(frames) == 3:  # We have frames from all videos
                # Resize frames to ensure same dimensions
                frames = {idx: cv2.resize(frame, (width, height)) 
                         for idx, frame in frames.items()}

                # Combine frames horizontally in correct order
                combined_frame = np.hstack([frames[1], frames[2], frames[3]])
                
                # Save frame
                frame_path = os.path.join(temp_dir, f'frame_{frame_count:06d}.png')
                cv2.imwrite(frame_path, combined_frame)
                frame_count += 1

                # Print progress every 100 frames
                if frame_count % 100 == 0:
                    print(f"Processed {frame_count} frames...")

        # Release video captures
        for cap_info in caps.values():
            cap_info['cap'].release()

        print(f"Total frames processed: {frame_count}")

        if frame_count == 0:
            raise RuntimeError("No frames were processed!")

        # Ensure output directory exists
        output_dir = os.path.dirname(os.path.abspath(output_video))
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print("Combining frames into video...")
        
        # Combine frames into video using ffmpeg
        (
            ffmpeg
            .input(os.path.join(temp_dir, 'frame_%06d.png'), pattern_type='sequence', framerate=fps)
            .output(output_video, 
                   vcodec='prores_ks',
                   pix_fmt='yuv422p10le',
                   preset='standard',
                   **{'profile:v': '3'})
            .overwrite_output()
            .run(capture_stdout=True, capture_stderr=True)
        )

        print(f"Video saved as {output_video}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

    finally:
        # Clean up temporary directory
        print("Cleaning up temporary files...")
        shutil.rmtree(temp_dir)

# Example usage
if __name__ == "__main__":
    video1_path = "data/video1.mov"
    video2_path = "data/video2.mov"
    video3_path = "data/video3.mov"  # Make sure this is a different video
    output_path = "data/combined_video.mov"
    
    try:
        combine_videos_parallel(video1_path, video2_path, video3_path, output_path)
    except Exception as e:
        print(f"An error occurred: {str(e)}")


# Cleanup 

Please clean up the resources 